### Modify garch2.ipynb

In [1]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://localhost',
                       connect_args={'read_default_file': '~/.mytest.cnf'})
conn = engine.connect()

In [2]:
def query_symbol(sym, col, conn):
    qry = pd.read_sql_query(
        '''select tradedate, ''' + col + ''' from USEQ_HIST
        where symbol="''' + sym + '''"
        and volume>0
         order by tradedate''',
        conn,
        index_col='tradedate'
    )
    return pd.DataFrame(qry, columns=[col])

In [3]:
dfcl = query_symbol("MS", "pct_close", conn)
dfcl.head()

,pct_close
tradedate,
2016-01-04,-1.03741
2016-01-05,-0.63532
2016-01-06,-2.49361
2016-01-07,-4.98361
2016-01-08,-2.07039


In [4]:
dfop = query_symbol("MS", "pct_open", conn)
dfop.head()

,pct_open
tradedate,
2016-01-04,-3.48946
2016-01-05,2.54071
2016-01-06,-2.70013
2016-01-07,-2.44858
2016-01-08,-1.57296


In [5]:
from arch import arch_model

#### GJR-Garch with Student's T

In [6]:
am = arch_model(dfcl, p=1, o=1, q=1, dist="StudentsT")
res = am.fit(update_freq=0)
print(res.summary())
print("likelihood ",res.loglikelihood)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2807.591628440884
            Iterations: 18
            Function evaluations: 160
            Gradient evaluations: 18
                      Constant Mean - GJR-GARCH Model Results                       
Dep. Variable:                    pct_close   R-squared:                       0.000
Mean Model:                   Constant Mean   Adj. R-squared:                  0.000
Vol Model:                        GJR-GARCH   Log-Likelihood:               -2807.59
Distribution:      Standardized Student's t   AIC:                           5627.18
Method:                  Maximum Likelihood   BIC:                           5658.82
                                              No. Observations:                 1440
Date:                      Mon, Sep 27 2021   Df Residuals:                     1439
Time:                              13:53:00   Df Model:                            1
                        

In [7]:
am = arch_model(dfop, p=1, o=1, q=1, dist="StudentsT")
res = am.fit(update_freq=0)
print(res.summary())
print("likelihood ",res.loglikelihood)




Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2846.398218171269
            Iterations: 11
            Function evaluations: 103
            Gradient evaluations: 11
                      Constant Mean - GJR-GARCH Model Results                       
Dep. Variable:                     pct_open   R-squared:                       0.000
Mean Model:                   Constant Mean   Adj. R-squared:                  0.000
Vol Model:                        GJR-GARCH   Log-Likelihood:               -2846.40
Distribution:      Standardized Student's t   AIC:                           5704.80
Method:                  Maximum Likelihood   BIC:                           5736.43
                                              No. Observations:                 1440
Date:                      Mon, Sep 27 2021   Df Residuals:                     1439
Time:                              13:53:00   Df Model:                            1
                        